In [1]:
from model import *
from attention import pHLA_attns_draw_save
from mutation import *
from utils import Logger, cut_peptide_to_specific_length

In [2]:
import math
from sklearn import metrics
from sklearn import preprocessing
import numpy as np
import pandas as pd
import re
import time
import datetime
import random
random.seed(1234)
from scipy import interp
import warnings
warnings.filterwarnings("ignore")

from collections import Counter
from collections import OrderedDict
from functools import reduce
from tqdm import tqdm, trange
from copy import deepcopy

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

import difflib

seed = 19961231
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

import os
os.chdir('./webserver/')
import argparse
import logging
import sys

In [3]:
parser = argparse.ArgumentParser(usage = 'peptide-HLA-I binding prediction')
parser.add_argument('--peptide_file', type = str, help = 'the path of the .fasta file contains peptides')
parser.add_argument('--HLA_file', type = str, help = 'the path of the .fasta file contains sequence')
parser.add_argument('--threshold', type = float, default = 0.5, help = 'the threshold to define predicted binder, float from 0 - 1, the recommended value is 0.5')
parser.add_argument('--cut_peptide', type = bool, default = False, help = 'Whether to split peptides larger than cut_length?')
parser.add_argument('--cut_length', type = int, default = 9, help = 'if there is a peptide sequence length > 15, we will segment the peptide according the length you choose, from 8 - 15')
parser.add_argument('--output_attention', type = bool, default = False, help = 'Output the mutual influence of peptide and HLA on the binding?')
parser.add_argument('--output_heatmap', type = bool, default = False, help = 'Visualize the mutual influence of peptide and HLA on the binding?')
parser.add_argument('--output_mutation', type = bool, default = True, help = 'Whether to perform mutations with better affinity for each sample?')
parser.add_argument('--output_dir', type = str, help = 'The directory where the output results are stored.')
args = parser.parse_args(args = [])
args

Namespace(HLA_file=None, cut_length=9, cut_peptide=False, output_attention=False, output_dir=None, output_heatmap=False, output_mutation=True, peptide_file=None, threshold=0.5)

# 读取文件

In [4]:
all_corrected_data = pd.read_csv('/home/chujunyi/5_ZY_MHC/results/pHLAIformer_all_corrected_data_res_pd.csv', index_col = 0)
all_corrected_data.head()

,peptide,length,HLA,label,HLA_sequence,y_true,y_pred,y_prob
0,QRYLDELMKL,10,HLA-B*27:08,1,YHTEYREICAKTDESNLYLNYHDYTWAVLAYEWY,1,1,9.999995e-01
1,TEIPVAKI,8,HLA-B*49:01,1,YHTKYREISTNTYENIAYWRYNLYTWAELAYLWY,1,1,1.000000e+00
2,DNTPSEMPP,9,HLA-A*11:01,0,YYAMYQENVAQTDVDTLYIIYRDYTWAAQAYRWY,0,0,1.262908e-07
3,ARLPGVSRSGFS,12,HLA-B*27:05,1,YHTEYREICAKTDEDTLYLNYHDYTWAVLAYEWY,1,1,9.999346e-01
4,HLVDFQVTI,9,HLA-A*02:02,1,YFAMYGEKVAHTHVDTLYLRYHYYTWAVWAYTWY,1,1,9.999999e-01


In [10]:
predict_list = []
for length in trange(8, 15):
    for hla in list(set(all_corrected_data[all_corrected_data.length == length][all_corrected_data.label == 0].HLA)):
        temp = all_corrected_data[all_corrected_data.length == length][all_corrected_data.label == 0][all_corrected_data.HLA == hla]
        if temp.shape[0] == 0: continue
        for idx in range(10):
            predict_list.append([temp.iloc[idx].HLA, temp.iloc[idx].HLA_sequence, temp.iloc[idx].peptide])
            
predict_data = pd.DataFrame(predict_list, columns = ['HLA', 'HLA_sequence', 'peptide'])
predict_data['length'] = [len(pep) for pep in predict_data.peptide]
predict_data

100%|██████████| 7/7 [00:25<00:00,  3.61s/it]


,HLA,HLA_sequence,peptide,length
0,HLA-B*51:01,YYATYRNIFTNTYENIAYWTYNYYTWAELAYLWH,TLTTIDLQ,8
1,HLA-B*51:01,YYATYRNIFTNTYENIAYWTYNYYTWAELAYLWH,LKKRCKDC,8
2,HLA-B*51:01,YYATYRNIFTNTYENIAYWTYNYYTWAELAYLWH,PMAGILEY,8
3,HLA-B*51:01,YYATYRNIFTNTYENIAYWTYNYYTWAELAYLWH,VDNLMCDG,8
4,HLA-B*51:01,YYATYRNIFTNTYENIAYWTYNYYTWAELAYLWH,DLLPETPW,8
...,...,...,...,...
3655,HLA-C*05:01,YYAGYREKYRQTDVNKLYLRYNFYTWAERAYTWY,DKETVLKGQIQAVA,14
3656,HLA-C*05:01,YYAGYREKYRQTDVNKLYLRYNFYTWAERAYTWY,GPGKNLPFSVENKW,14
3657,HLA-C*05:01,YYAGYREKYRQTDVNKLYLRYNFYTWAERAYTWY,LLSGSMEPAFHRGY,14
3658,HLA-C*05:01,YYAGYREKYRQTDVNKLYLRYNFYTWAERAYTWY,EFVKLAELPCLEDL,14


In [11]:
predict_data, predict_pep_inputs, predict_hla_inputs, predict_loader = read_predict_data(predict_data, batch_size)

use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")

model_file = 'pHLAIformer.pkl'

model_eval = Transformer().to(device)
model_eval.load_state_dict(torch.load(model_file), strict = True)

model_eval.eval()
y_pred, y_prob, attns = eval_step(model_eval, predict_loader, args.threshold, use_cuda)

predict_data['y_pred'], predict_data['y_prob'] = y_pred, y_prob
predict_data = predict_data.round({'y_prob': 4})

predict_data = predict_data[predict_data.y_pred == 0]
predict_data.shape

  0%|          | 0/4 [00:00<?, ?it/s]

# Samples =  3660


100%|██████████| 4/4 [00:00<00:00,  7.87it/s]


(3660, 6)

In [12]:
len(attns)

3660

In [17]:
3660*2

7320

# 突变

In [13]:
def curl_strs(method, mut_data, length, hla, ori_peptide, output_name):
    s1 = 'curl --data "method=' + method
    s2 = '&sequence_text=' + '%3E' + 'peptide' + str(1) + '%0A' + ori_peptide + '%0A'
    s3 = '&allele=' + hla
    s4 = '&length=' + str(length)
    s5 = '" http://tools-cluster-interface.iedb.org/tools_api/mhci/'
    
    output_pwd = '/home/chujunyi/5_ZY_MHC/results/mutation/NetMHCpan_BA/v2_output/'
    s6 = ' > ' + output_pwd + output_name + '\nsleep 1s'
        
    for idx in range(mut_data.shape[0]):
        peptide = mut_data.iloc[idx,:].mutation_peptide
        if idx != len(mut_data)-1:
            s2 += '%3E' + 'peptide' + str(idx+2) + '%0A' + peptide + '%0A'
        else:
            s2 += '%3E' + 'peptide' + str(idx+2) + '%0A' + peptide
    s = s1 + s2 + s3 + s4 + s5 + s6
    return s

In [14]:
def curl_sh_file(mut_data, hla, length, ori_peptide, method = 'netmhcpan_ba'):
    sh_str = ''
    output_name = '{}_{}_{}.output'.format(hla, length, ori_peptide)
    s = curl_strs(method, mut_data, length, hla, ori_peptide, output_name)
    sh_str += s + '\n'
            
    with open('/home/chujunyi/5_ZY_MHC/results/mutation/NetMHCpan_BA/v2_curl.sh', 'a') as f:
        f.write(sh_str)

In [15]:
mut_savepath = '/home/chujunyi/5_ZY_MHC/results/mutation/NetMHCpan_BA/'

success_length_hla_num, success_mut_num = dict(), dict()
for idx in trange(predict_data.shape[0]):
    peptide = predict_data.iloc[idx].peptide
    hla = predict_data.iloc[idx].HLA
    length = len(peptide)

    if len(peptide) < 8 or len(peptide) > 14: continue

    mut_peptides_df = pHLA_mutation_peptides(predict_data, attns, hla = hla, peptide = peptide, print_ = False)
    mut_data, _, _, mut_loader = read_predict_data(mut_peptides_df, batch_size)

    model_eval = Transformer().to(device)
    model_eval.load_state_dict(torch.load(model_file), strict = True)

    model_eval.eval()
    y_pred, y_prob, _ = eval_step(model_eval, mut_loader, args.threshold, use_cuda)

    mut_data['y_pred'], mut_data['y_prob'] = y_pred, y_prob
    mut_data = mut_data.round({'y_prob': 4})
#     mut_data.to_csv(mut_savepath + '{}_{}_mutation.csv'.format(hla, peptide), index = False)

    success_mut_data = mut_data[mut_data['y_pred'] == 1]
    curl_sh_file(success_mut_data, hla, length, peptide, method = 'netmhcpan_ba')
    
    if success_mut_data.shape[0] != 0:
        success_length_hla_num.setdefault(length, {})
        success_length_hla_num[length].setdefault(hla, 0)
        success_length_hla_num[length][hla] += 1
        
        min_mut_num = success_mut_data.mutation_AA_number.min()
        success_mut_num.setdefault(min_mut_num, 0)
        success_mut_num[min_mut_num] += 1


100%|██████████| 9/9 [00:00<00:00, 35.50it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

# Samples =  234



100%|██████████| 9/9 [00:00<00:00, 36.60it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

# Samples =  270



100%|██████████| 9/9 [00:00<00:00, 34.03it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

...

# Samples =  316


100%|██████████| 9/9 [00:00<00:00, 116.92it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

# Samples =  358


100%|██████████| 9/9 [00:00<00:00, 115.42it/s]

100%|██████████| 3660/3660 [26:27<00:00,  2.31it/s]

# Samples =  330


In [16]:
sum(success_mut_num.values()), success_mut_num, success_length_hla_num

(3633,
 {2: 1536, 1: 1692, 3: 350, 4: 55},
 {8: {'HLA-B*51:01': 10,
   'HLA-C*17:01': 10,
   'HLA-B*18:01': 10,
   'HLA-B*27:05': 10,
   'HLA-B*57:01': 10,
   'HLA-C*06:02': 10,
   'HLA-B*39:24': 10,
   'HLA-C*03:03': 10,
   'HLA-C*16:01': 10,
   'HLA-B*39:01': 10,
   'HLA-A*01:01': 10,
   'HLA-B*14:02': 10,
   'HLA-C*14:02': 10,
   'HLA-B*58:01': 10,
   'HLA-C*04:01': 10,
   'HLA-B*27:09': 10,
   'HLA-B*37:01': 10,
   'HLA-C*12:03': 10,
   'HLA-A*24:02': 10,
   'HLA-B*44:02': 10,
   'HLA-C*15:02': 9,
   'HLA-C*01:02': 10,
   'HLA-B*57:03': 10,
   'HLA-B*08:01': 10,
   'HLA-B*15:01': 10,
   'HLA-C*07:02': 10,
   'HLA-C*02:02': 10,
   'HLA-B*44:03': 10,
   'HLA-B*46:01': 10,
   'HLA-A*03:01': 10,
   'HLA-C*08:02': 10,
   'HLA-B*07:02': 10,
   'HLA-B*13:02': 10,
   'HLA-B*49:01': 10,
   'HLA-A*29:02': 10,
   'HLA-A*02:01': 10,
   'HLA-B*51:08': 10,
   'HLA-B*52:01': 10,
   'HLA-C*05:01': 10,
   'HLA-C*03:04': 10,
   'HLA-C*07:01': 10,
   'HLA-C*07:04': 10,
   'HLA-B*18:03': 10,
   'HLA-B

# NetMHCpan_BA结果

In [90]:
netmhcpan_pwd = '/home/chujunyi/5_ZY_MHC/results/mutation/NetMHCpan_BA/v2_output/'
ic50_file_list = os.listdir(netmhcpan_pwd)
ic50_file_list

['HLA-C*04:01_8_RKKLVIVG.output',
 'HLA-C*03:03_10_WMLEVTLDYT.output',
 'HLA-C*05:01_12_RASLAQGFTEDQ.output',
 'HLA-B*57:03_12_GYMTDTGTVEPL.output',
 'HLA-B*27:09_14_GAREEPTVQQAQRR.output',
 'HLA-B*07:02_8_QTFNHSGI.output',
 'HLA-B*40:02_12_LVHGTVVTYQCY.output',
 'HLA-A*02:19_9_LSFSSGSII.output',
 'HLA-B*52:01_9_KLRKLELSD.output',
 'HLA-A*30:01_9_QQQQPQQKT.output',
 'HLA-C*05:01_8_ASALYHFS.output',
 'HLA-B*27:09_14_EPVWTAYNRPADHF.output',
 'HLA-B*44:03_10_KKGAECFEEL.output',
 'HLA-C*07:01_9_VETEIEILK.output',
 'HLA-B*35:03_9_PQAPPYTDA.output',
 'HLA-B*08:01_11_LPARGKTYISK.output',
 'HLA-B*57:03_13_AIKYFGDIISVGQ.output',
 'HLA-B*46:01_10_VGIVLIIYFR.output',
 'HLA-C*04:01_13_EGEELKPEDKDPR.output',
 'HLA-B*39:01_8_QEKWVGYR.output',
 'HLA-A*23:01_11_LIQPMALAVRL.output',
 'HLA-A*68:02_14_LVYNLVYVEKISFL.output',
 'HLA-B*58:01_9_QHEDRCLRL.output',
 'HLA-B*44:03_12_LLVFTVALLVRG.output',
 'HLA-A*80:01_9_LLPPPPALC.output',
 'HLA-A*02:07_11_SISETLEEKPD.output',
 'HLA-A*02:04_11_VSFWVKFEQGD.output

In [97]:
error_files = []
for filename in tqdm(ic50_file_list):
    try:
        temp = pd.read_csv(netmhcpan_pwd + filename, sep = '\t')
        if temp.shape[1] != 10: 
            error_files.append(filename)
    except:
        error_files.append(filename)
error_files

100%|██████████| 3660/3660 [00:06<00:00, 606.15it/s]


[]

In [98]:
for filename in tqdm(error_files):
    temp = pd.read_csv(netmhcpan_pwd + filename, sep = '\t')
    print(filename, temp)

0it [00:00, ?it/s]


In [96]:
# with open('/home/chujunyi/5_ZY_MHC/results/mutation/NetMHCpan_BA/v2_curl.sh', 'r') as f:
#     sh_file = f.readlines()
    
# err_sh = ''
# for item in tqdm(sh_file):
#     for err in error_files:
#         if err in item:
#              err_sh += item + 'sleep 1s\n'
# with open('/home/chujunyi/5_ZY_MHC/results/mutation/NetMHCpan_BA/v2_curl_err3.sh', 'a') as f:
#     f.write(err_sh)

100%|██████████| 7320/7320 [00:00<00:00, 731704.13it/s]


In [99]:
ic50_success_num, rank_success_num, ic50_rank_success_num = 0, 0, 0
err_num = 0
for filename in tqdm(ic50_file_list):
    try:
        temp = pd.read_csv(netmhcpan_pwd + filename, sep = '\t')
    
        if temp[temp.ic50 <=500].shape[0] > 0:
            ic50_success_num += 1
        if  temp[temp.percentile_rank <= 2].shape[0] > 0:
            rank_success_num += 1
        if temp[temp.ic50 <=500].shape[0] > 0 or temp[temp.percentile_rank <= 2].shape[0] > 0:
            ic50_rank_success_num += 1
    except:
        err_num += 1
ic50_success_num, rank_success_num, ic50_rank_success_num, err_num

100%|██████████| 3660/3660 [00:10<00:00, 351.80it/s]


(2905, 3418, 3419, 0)

In [104]:
2905/3630, 3418/3630, 3419/3630

(0.800275482093664, 0.9415977961432507, 0.9418732782369146)

# 画径向图的例子

In [82]:
for idx in trange(predict_data.shape[0]):
    peptide = predict_data.iloc[idx].peptide
    hla = predict_data.iloc[idx].HLA
    length = len(peptide)

    example_df = pHLA_mutation_peptides(predict_data, attns, hla = hla, peptide = peptide, print_ = False)
    example_data, _, _, example_loader = read_predict_data(example_df, batch_size)

    model_eval = Transformer().to(device)
    model_eval.load_state_dict(torch.load(model_file), strict = True)

    model_eval.eval()
    example_y_pred, example_y_prob, _ = eval_step(model_eval, example_loader, args.threshold, use_cuda)

    example_data['y_pred'], example_data['y_prob'] = example_y_pred, example_y_prob
    example_data = example_data.round({'y_prob': 4})
    try:
        example_ic50 = pd.read_csv(netmhcpan_pwd + '{}_{}_{}.output'.format(hla, length, peptide), sep = '\t')
    except:
        continue
    merged_data = pd.merge(example_ic50, example_data, how = 'inner', left_on = 'peptide', right_on = 'mutation_peptide')
    merged_data = merged_data[['HLA', 'original_peptide', 'mutation_peptide', 'length', 'ic50', 'y_pred', 'y_prob', 'percentile_rank', 'mutation_position_AAtype', 'mutation_AA_number']]

    if merged_data[merged_data.mutation_AA_number == 1].shape[0] > 2:
        print(hla, peptide, length)
        break

100%|██████████| 9/9 [00:00<00:00, 25.95it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

# Samples =  234



100%|██████████| 9/9 [00:00<00:00, 37.16it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

# Samples =  270



100%|██████████| 9/9 [00:00<00:00, 35.69it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

# Samples =  216



100%|██████████| 9/9 [00:00<00:00, 36.52it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

# Samples =  216



100%|██████████| 9/9 [00:00<00:00, 36.09it/s]

  0%|          | 4/3660 [00:02<32:04,  1.90it/s]

# Samples =  207
HLA-B*51:01 DLLPETPW 8


In [106]:
peptide = 'DLLPETPW'
hla = 'HLA-B*51:01'
length = len(peptide)

example_df = pHLA_mutation_peptides(predict_data, attns, hla = hla, peptide = peptide, print_ = True)
example_data, _, _, example_loader = read_predict_data(example_df, batch_size)

model_eval = Transformer().to(device)
model_eval.load_state_dict(torch.load(model_file), strict = True)

model_eval.eval()
example_y_pred, example_y_prob, _ = eval_step(model_eval, example_loader, args.threshold, use_cuda)

example_data['y_pred'], example_data['y_prob'] = example_y_pred, example_y_prob
example_data = example_data.round({'y_prob': 4})

example_ic50 = pd.read_csv(netmhcpan_pwd + '{}_{}_{}.output'.format(hla, length, peptide), sep = '\t')

merged_data = pd.merge(example_ic50, example_data, how = 'inner', left_on = 'peptide', right_on = 'mutation_peptide')
merged_data = merged_data[['HLA', 'original_peptide', 'mutation_peptide', 'length', 'ic50', 'y_pred', 'y_prob', 'percentile_rank', 'mutation_position_AAtype', 'mutation_AA_number']]

Counter(example_data.y_pred), Counter(merged_data.y_pred)

100%|██████████| 1/1 [00:00<00:00, 28.91it/s]

********** Strategy 1 **********
[1, 2, 5, 3, 8, 6, 7, 4] [1, 2, 3, 4, 5, 8, 6, 7]
Effect of 1D in Negative DLLPETPW < Effect of 1['D', 'L', 'I', 'Y', 'V', 'F', 'M'] in Positive HLA-B*51:01, Replace to 1['D', 'L', 'I']
['DLLPETPW', 'LLLPETPW', 'ILLPETPW']
Effect of 2L in Negative DLLPETPW < Effect of 2['P', 'A'] in Positive HLA-B*51:01, Replace to 2['P', 'A']
['DPLPETPW', 'DALPETPW']
['LPLPETPW', 'LALPETPW']
['IPLPETPW', 'IALPETPW']
Effect of 5E in Negative DLLPETPW < Effect of 3['Y', 'P', 'F', 'L', 'A', 'V'] in Positive HLA-B*51:01, Replace to 3['Y', 'P', 'F']
['DPYPETPW', 'DPPPETPW', 'DPFPETPW']
['DAYPETPW', 'DAPPETPW', 'DAFPETPW']
['LPYPETPW', 'LPPPETPW', 'LPFPETPW']
['LAYPETPW', 'LAPPETPW', 'LAFPETPW']
['IPYPETPW', 'IPPPETPW', 'IPFPETPW']
['IAYPETPW', 'IAPPETPW', 'IAFPETPW']
Effect of 5E in Negative DLLPETPW < Effect of 4['L', 'V', 'S', 'I', 'P', 'A', 'R', 'T', 'K', 'E'] in Positive HLA-B*51:01, Replace to 4['L', 'V', 'S']
['DPYLETPW', 'DPYVETPW', 'DPYSETPW']
['DPPLETPW', 'DPPVETPW

(Counter({0: 5, 1: 202}), Counter({1: 202, 0: 1}))

In [158]:
temp_rank = merged_data[merged_data.percentile_rank <= 2]
temp_ic50 = merged_data[merged_data.ic50 <= 500]
temp = pd.concat([temp_rank, temp_ic50], axis = 0).drop_duplicates().sort_values('mutation_AA_number').reset_index(drop = True)
temp

,HLA,original_peptide,mutation_peptide,length,ic50,y_pred,y_prob,percentile_rank,mutation_position_AAtype,mutation_AA_number
0,HLA-B*51:01,DLLPETPW,DAYPETPW,8,6441.13,1,0.9942,0.91,"2|L/A,3|L/Y",2
1,HLA-B*51:01,DLLPETPW,DALPETPL,8,5018.13,1,0.9997,0.67,"2|L/A,8|W/L",2
2,HLA-B*51:01,DLLPETPW,DPLPETPV,8,4203.77,1,1.0000,0.55,"2|L/P,8|W/V",2
3,HLA-B*51:01,DLLPETPW,DAFPETPW,8,7539.55,1,0.9962,1.20,"2|L/A,3|L/F",2
4,HLA-B*51:01,DLLPETPW,DPYPETPW,8,10453.66,1,0.9997,1.80,"2|L/P,3|L/Y",2
5,HLA-B*51:01,DLLPETPW,LPLPETPW,8,8659.59,1,1.0000,1.50,"1|D/L,2|L/P",2
6,HLA-B*51:01,DLLPETPW,DALPETPV,8,1197.02,1,1.0000,0.14,"2|L/A,8|W/V",2
7,HLA-B*51:01,DLLPETPW,DALPETPI,8,647.16,1,1.0000,0.08,"2|L/A,8|W/I",2
8,HLA-B*51:01,DLLPETPW,DPLPETPL,8,10251.27,1,1.0000,1.80,"2|L/P,8|W/L",2
9,HLA-B*51:01,DLLPETPW,DPLPETPI,8,2716.45,1,1.0000,0.33,"2|L/P,8|W/I",2


In [159]:
example_mut_posi_aa = ['2|L/A', '2|L/P', '8|W/V', '8|W/I', '8|W/L', '3|L/Y', '3|L/P', '3|L/F', '1|D/L', '1|D/I']

example_index = []
for item in temp.mutation_position_AAtype:
    item = item.split(',')
    for m in item:
        if m not in example_mut_posi_aa:
            break
    else:
        example_index.append(item)
pd.Series(example_index)

0                   [2|L/A, 3|L/Y]
1                   [2|L/A, 8|W/L]
2                   [2|L/P, 8|W/V]
3                   [2|L/A, 3|L/F]
4                   [2|L/P, 3|L/Y]
5                   [1|D/L, 2|L/P]
6                   [2|L/A, 8|W/V]
7                   [2|L/A, 8|W/I]
8                   [2|L/P, 8|W/L]
9                   [2|L/P, 8|W/I]
10                  [1|D/I, 2|L/P]
11           [2|L/A, 3|L/Y, 8|W/L]
12           [2|L/P, 3|L/Y, 8|W/V]
13           [1|D/I, 2|L/P, 3|L/F]
14           [1|D/L, 2|L/A, 8|W/I]
15           [1|D/I, 2|L/P, 8|W/V]
16           [1|D/I, 2|L/P, 8|W/L]
17           [2|L/P, 3|L/F, 8|W/V]
18           [1|D/I, 2|L/A, 8|W/V]
19           [2|L/A, 3|L/P, 8|W/V]
20           [1|D/L, 2|L/A, 8|W/V]
21           [1|D/L, 2|L/P, 3|L/F]
22           [2|L/P, 3|L/Y, 8|W/L]
23           [1|D/L, 2|L/P, 3|L/Y]
24           [1|D/I, 2|L/P, 3|L/Y]
25           [2|L/P, 3|L/F, 8|W/L]
26           [1|D/L, 2|L/P, 8|W/L]
27           [2|L/A, 3|L/P, 8|W/I]
28           [1|D/I,

In [160]:
example_indexs = []
for item in example_index:
    temp = ['0','0','0','0']
    for m in item:
        if m[0] == '2':
            temp[0] = m
        elif m[0] == '8':
            temp[1] = m
        elif m[0] == '3':
            temp[2] = m
        elif m[0] == '1':
            temp[3] = m
    num_no0 = 4 - len([i for i in temp if i == '0'])
    for i in range(num_no0):
        if temp[i] == '0':
            break
    else:
        example_indexs.append(temp)
example_indexs

[['2|L/A', '8|W/L', '0', '0'],
 ['2|L/P', '8|W/V', '0', '0'],
 ['2|L/A', '8|W/V', '0', '0'],
 ['2|L/A', '8|W/I', '0', '0'],
 ['2|L/P', '8|W/L', '0', '0'],
 ['2|L/P', '8|W/I', '0', '0'],
 ['2|L/A', '8|W/L', '3|L/Y', '0'],
 ['2|L/P', '8|W/V', '3|L/Y', '0'],
 ['2|L/P', '8|W/V', '3|L/F', '0'],
 ['2|L/A', '8|W/V', '3|L/P', '0'],
 ['2|L/P', '8|W/L', '3|L/Y', '0'],
 ['2|L/P', '8|W/L', '3|L/F', '0'],
 ['2|L/A', '8|W/I', '3|L/P', '0'],
 ['2|L/A', '8|W/L', '3|L/F', '0'],
 ['2|L/P', '8|W/I', '3|L/P', '0'],
 ['2|L/P', '8|W/I', '3|L/F', '0'],
 ['2|L/A', '8|W/V', '3|L/Y', '0'],
 ['2|L/A', '8|W/I', '3|L/F', '0'],
 ['2|L/A', '8|W/V', '3|L/F', '0'],
 ['2|L/P', '8|W/V', '3|L/P', '0'],
 ['2|L/A', '8|W/I', '3|L/Y', '0'],
 ['2|L/A', '8|W/L', '3|L/P', '0'],
 ['2|L/P', '8|W/I', '3|L/Y', '0'],
 ['2|L/A', '8|W/I', '3|L/P', '1|D/L'],
 ['2|L/A', '8|W/V', '3|L/P', '1|D/I'],
 ['2|L/A', '8|W/V', '3|L/P', '1|D/L'],
 ['2|L/P', '8|W/L', '3|L/P', '1|D/L'],
 ['2|L/A', '8|W/I', '3|L/P', '1|D/I'],
 ['2|L/P', '8|W/L', '3|L

In [166]:
pd.Series(sorted(example_indexs))

0             [2|L/A, 8|W/I, 0, 0]
1         [2|L/A, 8|W/I, 3|L/F, 0]
2     [2|L/A, 8|W/I, 3|L/F, 1|D/I]
3     [2|L/A, 8|W/I, 3|L/F, 1|D/L]
4         [2|L/A, 8|W/I, 3|L/P, 0]
5     [2|L/A, 8|W/I, 3|L/P, 1|D/I]
6     [2|L/A, 8|W/I, 3|L/P, 1|D/L]
7         [2|L/A, 8|W/I, 3|L/Y, 0]
8     [2|L/A, 8|W/I, 3|L/Y, 1|D/I]
9     [2|L/A, 8|W/I, 3|L/Y, 1|D/L]
10            [2|L/A, 8|W/L, 0, 0]
11        [2|L/A, 8|W/L, 3|L/F, 0]
12    [2|L/A, 8|W/L, 3|L/F, 1|D/I]
13    [2|L/A, 8|W/L, 3|L/F, 1|D/L]
14        [2|L/A, 8|W/L, 3|L/P, 0]
15        [2|L/A, 8|W/L, 3|L/Y, 0]
16    [2|L/A, 8|W/L, 3|L/Y, 1|D/I]
17    [2|L/A, 8|W/L, 3|L/Y, 1|D/L]
18            [2|L/A, 8|W/V, 0, 0]
19        [2|L/A, 8|W/V, 3|L/F, 0]
20    [2|L/A, 8|W/V, 3|L/F, 1|D/I]
21    [2|L/A, 8|W/V, 3|L/F, 1|D/L]
22        [2|L/A, 8|W/V, 3|L/P, 0]
23    [2|L/A, 8|W/V, 3|L/P, 1|D/I]
24    [2|L/A, 8|W/V, 3|L/P, 1|D/L]
25        [2|L/A, 8|W/V, 3|L/Y, 0]
26    [2|L/A, 8|W/V, 3|L/Y, 1|D/I]
27    [2|L/A, 8|W/V, 3|L/Y, 1|D/L]
28            [2|L/P

In [109]:
pd.set_option('display.max_rows', 500)

# 相似性同源性统计

In [174]:
import difflib

In [216]:
def cal_mutate_num(original_peptide, mut_peptide):
    pep_length = len(original_peptide)
    mutate_num = 0
    for a, b in zip(original_peptide, mut_peptide):
        if a != b: mutate_num += 1
#     print(original_peptide)
#     print(mut_peptide, mutate_num)
    return mutate_num

In [222]:
mut_num_similarity_dict = dict()
for filename in tqdm(ic50_file_list):
    peptide = filename.split('_')[2][:-7]
    try:
        temp = pd.read_csv(netmhcpan_pwd + filename, sep = '\t')
        temp['original_peptide'] = peptide
        temp['similarity'] = [difflib.SequenceMatcher(None, item ,peptide).ratio() for item in temp.peptide]
        temp['mut_num'] = [cal_mutate_num(peptide, item) for item in temp.peptide]
        for mut_n, s in zip(temp.mut_num, temp.similarity):
            mut_num_similarity_dict.setdefault(mut_n, [])
            mut_num_similarity_dict[mut_n].append(s)
    except:
        continue

100%|██████████| 3660/3660 [00:20<00:00, 176.93it/s]


In [224]:
for n in [1, 2, 3, 4]:
    print(n, len(mut_num_similarity_dict[n]), np.mean(mut_num_similarity_dict[n]))

1 3881 0.899583392787464
2 32735 0.7985008530495014
3 153766 0.7032671613902185
4 362239 0.6116073937276928
